In [1]:
import numpy as np
from scipy.stats import ortho_group
import sys
sys.path.append("../")
import numpy as np 
from utils import *
from lattice import *

In [2]:
bd = 2
sps = 3
B = FF.block1D(bd, sps, bd, canonical=True, seed = 9).real
B_transfer = np.einsum("ipj, kpl->ikjl",B, B).reshape((bd**2,)*2)
Et, Vt = np.linalg.eig(B_transfer)
print("spectrum of transfer matrix of canonical MPS : ", Et)
X = Vt[:,0].reshape(bd, bd)
print("fixed point is identiy matrix : ",np.linalg.norm(X / X[0,0] - np.eye(X.shape[0])))

spectrum of transfer matrix of canonical MPS :  [ 1.          0.27222952  0.14273616 -0.18315069]
fixed point is identiy matrix :  1.9903230855597885e-15


In [12]:
B = FF.block1D(bd, sps, bd, canonical=False, seed = 9, sym=True).real
B_transfer = np.einsum("ipj, kpl->ikjl",B, B).reshape((bd**2,)*2)
Et, Vt = np.linalg.eig(B_transfer)
print("spectrum of transfer matrix of canonical MPS : ", Et)
X = Vt[:,0].reshape(bd, bd)
print("Eigen vector corresponds to the largest ev can be taken to be positive matrix\n", np.linalg.eigvals(X))

spectrum of transfer matrix of canonical MPS :  [ 4.22627733  1.15051744  0.6032426  -0.77404563]
Eigen vector corresponds to the largest ev can be taken to be positive matrix
 [-0.23269428 -0.97254993]


In [14]:
B = FF.block1D(bd, sps, bd, canonical=False, seed = 9, sym=False).real
B_transfer = np.einsum("ipj, kpl->ikjl",B, B).reshape((bd**2,)*2)
Et, Vt = np.linalg.eig(B_transfer)
print("spectrum of transfer matrix of canonical MPS : ", Et)
X = Vt[:,0].reshape(bd, bd)
print("Eigen vector corresponds to the largest ev can be taken to be positive matrix (non symmetric calse ) :\n", np.linalg.eigvals(X))

spectrum of transfer matrix of canonical MPS :  [ 3.10356847+0.j          1.08734681+0.29120278j  1.08734681-0.29120278j
 -0.07227035+0.j        ]
Eigen vector corresponds to the largest ev can be taken to be positive matrix (non symmetric calse ) :
 [-0.06641156+0.j -0.99779232+0.j]


**Check for multiple seeds and see if the eigenvalue of X is either all positive or negative**

If you take random transfer matrix, then you will never achieve this property. We'll see below the case if transfer matrix created randomly

In [201]:
bd = 2
seed = 10 # change here
S = np.zeros((bd,)*4)
for i in range(bd):
    for j in range(bd):
        S[i,j,j,i] = 1
S = S.reshape(bd**2, bd**2)
Es, Vs = np.linalg.eigh(S)


np.random.seed(seed)
U_tmp = np.zeros((bd**2,bd**2))
idx = np.argwhere(Es == -1)[-1]
U_tmp[:idx[0]+1, :idx[0]+1] = ortho_group.rvs(dim=np.round(idx[0]+1)) if idx[0] != 0 else 1
U_tmp[idx[0]+1:, idx[0]+1:] = ortho_group.rvs(dim=np.round(bd**2-idx[0]-1))
U = Vs @ U_tmp @ Vs.T

lam_tmp = np.sort(np.random.uniform(low=-0.5,high = 1, size = (bd**2)))
# lam_tmp[-1:] = 1
# lam_tmp[0] = 0
lam = Vs @ np.diag(lam_tmp) @ Vs.T
A_transfer = U @ lam @ U.T
print("check if this is symmetric : ",np.linalg.norm(S @ A_transfer @ S - A_transfer))

check if this is symmetric :  3.925231146709438e-17


In [206]:
lam_tmp = np.sort(np.random.uniform(low=-0.5,high = 1, size = (bd**2)))
lam_tmp[-2:] = 1
# lam_tmp[0] = 0
lam = Vs @ np.diag(lam_tmp) @ Vs.T
A_transfer = U @ lam @ U.T
print("check if this is symmetric : ",np.linalg.norm(S @ A_transfer @ S - A_transfer))

check if this is symmetric :  0.0


In [207]:
Et, Vt = np.linalg.eigh(A_transfer)
print("spectrum of transfer matrix of canonical MPS : ", Et)
X = Vt[:,-1].reshape(bd, bd)
print("Eigen vector corresponds to the largest ev can be taken to be positive matrix\n", np.linalg.eigvals(X))

spectrum of transfer matrix of canonical MPS :  [0.06001114 0.15102099 1.         1.        ]
Eigen vector corresponds to the largest ev can be taken to be positive matrix
 [-0.0429247   0.99907831]


**X not being positive matrix means this by no means can be represented in MPS form** 
When you shift the A_encoder so that the all energy become positive, then fixed point become positive again.

In [208]:
A_transfer_tmp = np.einsum("ijkl->ikjl",A_transfer.reshape((bd,)*4)).reshape(bd**2, bd**2)
Et_tmp, Vt_tmp = np.linalg.eigh(A_transfer_tmp)
print(Et_tmp)
A_transfer_tmp = A_transfer_tmp - 1 * Et_tmp[0] * np.eye(bd**2) # shift energy 

[-0.06072711  0.03366411  0.87926175  1.11735688]


In [209]:
A_transfer = np.einsum("ijkl->ikjl",A_transfer_tmp.reshape((bd,)*4)).reshape(bd**2, bd**2)
Et, Vt = np.linalg.eigh(A_transfer)
print("spectrum of transfer matrix of canonical MPS : ", Et)
X = Vt[:,-1].reshape(bd, bd)
print("Eigen vector corresponds to the largest ev can be taken to be positive matrix\n", np.linalg.eigvals(X))

spectrum of transfer matrix of canonical MPS :  [0.06001114 0.15292987 1.         1.11954534]
Eigen vector corresponds to the largest ev can be taken to be positive matrix
 [-0.78520565 -0.61923508]


**But note that this is not necessary condition. You don't need to shift by E[0], and indeed much smaller shift can result in positive matrix**


Important things for uniquness should be injectivity.

# Create canonical MPS with more than one block

In [417]:
sps = 3
bd1 = 1
bd2 = 1
bd =  bd1 + bd2
seed = 15
B1 = FF.block1D(bd2, sps, bd2, canonical=True, seed = seed)
B2 = FF.block1D(bd1, sps, bd1, canonical=True, seed = seed+1)

A = np.zeros((bd, sps, bd))

In [420]:
lams = np.random.uniform(low = 0,  high = 1, size= 2)
# lams[:] = 1
A[:bd1, :, :bd1] = B2 * lams[0]
A[bd1:, :, bd1:] = B1 * lams[1]

In [423]:
A_transfer = np.einsum("ipj, kpl->ikjl",A, A).reshape((bd**2,)*2)
E, V = np.linalg.eig(A_transfer)
idx = E.argsort()[::-1] 
E = E[idx]
V = V[:, idx]

print("spectrum of transfer matrix : ", E)
print("note that fixed point is not invertible : ",np.linalg.eigvals(V[:,0].reshape(bd,bd)))

spectrum of transfer matrix :  [ 0.47434373  0.02680798 -0.06560394 -0.06560394]
note that fixed point is not invertible :  [1. 0.]


In [424]:
B = A.transpose(1,0,2)
print("any lenght of set of B cannot be injective : ",np.array([0,0,1]) @ (B[0] @ B[1] @ B[1]) @ np.array([1,0,0]))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)

## Construct parent ham with above MPS

In [425]:
A2 = np.einsum("ijk,klm->jlim", A, A).reshape(sps**2, bd**2)
U, s, V = np.linalg.svd(A2)
Up = U[:, len(s):]
h = Up @ Up.T

In [426]:
L = 6
H = sum_ham(h, [[i,(i+1)%L] for i in range(L)], L, sps)

In [427]:
EH, VH = np.linalg.eigh(H)

In [428]:
print(EH)

[-3.70880987e-15  2.62230971e-15  2.73834641e-01  3.33621423e-01
  3.33621423e-01  3.49389368e-01  4.37404395e-01  4.37404395e-01
  5.28488897e-01  5.92540286e-01  5.92540286e-01  6.12446409e-01
  6.21512223e-01  6.21512223e-01  7.48313630e-01  7.77519614e-01
  7.77519614e-01  9.71406680e-01  9.99582482e-01  1.00890144e+00
  1.00890144e+00  1.04805711e+00  1.04805711e+00  1.07209867e+00
  1.15579175e+00  1.15579175e+00  1.16275147e+00  1.17073069e+00
  1.17073069e+00  1.22358717e+00  1.29481393e+00  1.29481393e+00
  1.29487311e+00  1.30065476e+00  1.30065476e+00  1.31077300e+00
  1.33538965e+00  1.33538965e+00  1.37085124e+00  1.39024828e+00
  1.39024828e+00  1.42170922e+00  1.42170922e+00  1.43852950e+00
  1.44003693e+00  1.44003693e+00  1.44997228e+00  1.48430535e+00
  1.48430535e+00  1.48598380e+00  1.48848320e+00  1.48848320e+00
  1.51727798e+00  1.53048173e+00  1.53048173e+00  1.55640318e+00
  1.55962261e+00  1.58149076e+00  1.58149076e+00  1.61282400e+00
  1.61282400e+00  1.61291